In [1]:
run instagram_poster

In [2]:
import re
import yaml
from datetime import date
from markdown2 import markdown
from mdplain import plain
from pathlib import Path

places = []
taste_labels = ["Do Not Recommend", "Something Going For It", "Good", "Phenomenal"]
taste_labels_short = ["dnr", "sgfi", "good", "phenomenal"]

def rating_to_formatting(rating, rating_labels):
    return rating_labels[rating]
def rating_html(rating, rating_labels):
    return "&nbsp;".join(
        [
            f'<span style="color: {color if rating == ix else faded_color}" aria-hidden="{"false" if rating == ix else "true"}">{label}</span>'
            for ix, (label, color) in enumerate(zip(rating_labels, rating_colors))
        ]
    )
def suffix(d):
    return "th" if 11 <= d <= 13 else {1: "st", 2: "nd", 3: "rd"}.get(d % 10, "th")
def custom_strftime(format_, t):
    return t.strftime(format_).replace("{S}", str(t.day) + suffix(t.day))
def format_visited(visited):
    return custom_strftime("{S} %B %Y", visited)
def format_blurb(md):
    return " ".join(plain(re.sub(r"\s+", " ", md.strip())).split(" ")[:50]) + "..."
    
for place_md in Path("../places").glob("*.md"):
    try: 
        slug = place_md.parts[-1][:-3]
        relative_url = f"/places/{slug}/"
        with open(place_md) as f:
            _, frontmatter, md = f.read().split("---", 2)
        meta = yaml.load(frontmatter, Loader=yaml.Loader)
        meta["url"] = relative_url
        meta["slug"] = slug
        visited = date.fromisoformat(meta["visited"])
        meta["visited_display"] = format_visited(visited)
        meta["review_age"] = (date.today() - visited).days
        meta["taste_label"] = rating_to_formatting(meta["taste"], taste_labels)
        meta["taste_label_short"] = rating_to_formatting(meta["taste"], taste_labels_short)
        meta["blurb"] = format_blurb(md)

        places.append(meta)

    except Exception as e:
        print(place_md.name, e)


In [3]:
def toggle_instagram_published(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    pattern = r'(instagram_published:\s*)(True|False)'

    def toggle_match(match):
        current_value = match.group(2)
        new_value = 'False' if current_value == 'True' else 'True'
        return match.group(1) + new_value 

    new_content, n = re.subn(pattern, toggle_match, content)
    if n > 0:
        with open(file_path, 'w') as file:
            file.write(new_content)
    
    

In [4]:
places = sorted(places, key=lambda x: -x['review_age'])
places = [p for p in places if p['instagram_published'] is False]
places_to_update = places[:10]
[p['name'] for p in places_to_update]

['Peña Pachamama',
 'Taqueria Cancun',
 'Ramenwell',
 "Rhea's Deli & Market",
 'Souvla',
 'Sunflower',
 "Reem's California Mission",
 'Dinosaurs',
 'House of Dim Sum (Mission)',
 'Al Carajo']

In [5]:
credentials_file_path = 'credentials.json'
bot = InstagramBot(credentials_file_path)
bot.login()

Message: 
Stacktrace:
#0 0x5648819a6dc3 <unknown>
#1 0x5648816954e7 <unknown>
#2 0x5648816e035d <unknown>
#3 0x5648816e0411 <unknown>
#4 0x564881723774 <unknown>
#5 0x56488170247d <unknown>
#6 0x564881720c29 <unknown>
#7 0x5648817021f3 <unknown>
#8 0x5648816d328a <unknown>
#9 0x5648816d3c5e <unknown>
#10 0x56488196b0eb <unknown>
#11 0x56488196f03b <unknown>
#12 0x564881957201 <unknown>
#13 0x56488196fba2 <unknown>
#14 0x56488193c0bf <unknown>
#15 0x564881995f18 <unknown>
#16 0x5648819960f0 <unknown>
#17 0x5648819a5f14 <unknown>
#18 0x7b4aaca94ac3 <unknown>



In [6]:
MINUTE = 60

In [ ]:
for place in places_to_update:
    try:
        bot.upload_post(place)
        filepath = Path(".." + place['url'][:-1] + ".md")
        toggle_instagram_published(filepath)
    except Exception as e:
        print(e)
    sleep_time = 30 * MINUTE * random()
    print(f"sleeping for {sleep_time / MINUTE} minutes")
    sleep(sleep_time)

sleeping for 27.806489665654478 minutes


In [60]:
# credentials_file_path = 'credentials.json'
# bot = InstagramBot(credentials_file_path)
# bot.login()
# for place in places_to_update:
#     bot.upload_post(place)
#     filepath = Path(".." + place['url'][:-1] + ".md")
#     toggle_instagram_published(filepath)
#     sleep(120 * random())
